<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Notebooks/18-2-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optimización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Andrés Merino &bull; Periodo 2025-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

Este notebook está diseñado como una guía introductoria para implementar optimización de hiperparámetros en modelos de aprendizaje automático.


Los paquetes necesarios son:

In [1]:
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización

from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier


---
## <font color='264CC7'> Ejemplo práctico </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero leamos los datos y seleccionemos las columnas que utilizaremos:

In [2]:
# Leer los datos
data = pd.read_csv('https://raw.githubusercontent.com/andres-merino/AprendizajeAutomaticoInicial-05-N0105/refs/heads/main/2-Notebooks/datos/Pokemon.csv')
# Tomo las columnas de interés
numeric_cols = ['Attack', 'Defense', 'Speed', 'Sp. Atk', 'Sp. Def', 'HP']
class_col = ['Stage']
data = data[['Name', *numeric_cols, *class_col]]
# Muestro los primeros registros
display(data.head())

,Name,Attack,Defense,Speed,Sp. Atk,Sp. Def,HP,Stage
0,Bulbasaur,49,49,45,65,65,45,2
1,Ivysaur,62,63,60,80,80,60,1
2,Venusaur,82,83,80,100,100,80,0
3,Charmander,52,43,65,60,50,39,2
4,Charmeleon,64,58,80,80,65,58,1


Revisemos los datos:

In [3]:
data.describe()

,Attack,Defense,Speed,Sp. Atk,Sp. Def,HP,Stage
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,72.549669,68.225166,68.933775,67.139073,66.019868,64.211921,0.569536
std,26.596162,26.916704,26.746880,28.534199,24.197926,28.590117,0.678331
min,5.000000,5.000000,15.000000,15.000000,20.000000,10.000000,0.000000
25%,51.000000,50.000000,46.500000,45.000000,49.000000,45.000000,0.000000
50%,70.000000,65.000000,70.000000,65.000000,65.000000,60.000000,0.000000
75%,90.000000,84.000000,90.000000,87.500000,80.000000,80.000000,1.000000
max,134.000000,180.000000,140.000000,154.000000,125.000000,250.000000,2.000000


Dividimos los datos en los conjuntos de entrenamiento y prueba.

In [4]:
X = data[numeric_cols]

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, data['Stage'], test_size=0.2, random_state=42, stratify=data['Stage'])

### <font color='264CC7'> Modelo </font>

Definimos el modelo:

In [5]:
# Crear y entrenar un arbol con ganancia de información
modelo_base = RandomForestClassifier(random_state=42)

# Parámetros del modelo
modelo_base.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

Definimos los parámetros y la grilla:

In [6]:
parametros = {'n_estimators': [10, 20], 
              'max_depth': [None, 3, 5]}	
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

modelo = GridSearchCV(modelo_base, parametros, cv=k_fold, scoring='accuracy')
modelo

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 3, 5], 'n_estimators': [10, 20]},
             scoring='accuracy')

Realizamos la búsqueda de hiperparámetros:

In [7]:
# Definir los k-folds
modelo.fit(X_train, y_train)

# Mostrar los resultados
pd.DataFrame(modelo.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011262,0.001796,0.001699,0.000510,None,10,"{'max_depth': None, 'n_estimators': 10}",0.666667,0.833333,0.708333,0.791667,0.583333,0.716667,0.088976,6
1,0.015104,0.000200,0.001699,0.000243,None,20,"{'max_depth': None, 'n_estimators': 20}",0.583333,0.833333,0.708333,0.833333,0.666667,0.725000,0.097183,4
2,0.007700,0.000244,0.001300,0.000245,3,10,"{'max_depth': 3, 'n_estimators': 10}",0.708333,0.916667,0.625000,0.833333,0.625000,0.741667,0.116070,1
3,0.014502,0.000314,0.001600,0.000201,3,20,"{'max_depth': 3, 'n_estimators': 20}",0.750000,0.833333,0.666667,0.833333,0.583333,0.733333,0.097183,2
4,0.007801,0.000244,0.001300,0.000246,5,10,"{'max_depth': 5, 'n_estimators': 10}",0.750000,0.833333,0.583333,0.833333,0.625000,0.725000,0.104083,3
5,0.015300,0.001119,0.001703,0.000247,5,20,"{'max_depth': 5, 'n_estimators': 20}",0.750000,0.791667,0.583333,0.875000,0.625000,0.725000,0.107367,4


Vemos los mejores hiperparámetros:

In [8]:
# Mejores parámetros
print("Mejores parámetros", modelo.best_params_)
print("Mejor score", modelo.best_score_)

Mejores parámetros {'max_depth': 3, 'n_estimators': 10}
Mejor score 0.7416666666666667


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
Prueba otros valores de <code>cv</code>, <code>scoring</code> y de los hiperparámetros para ver cómo cambian los resultados. 
</div>
</br>

Realizamos la predicción:

In [9]:
# Realizar predicciones y evaluar el modelo
y_pred = modelo.predict(X_test)

# Precisión del modelo con dos decimales
accuracy = round(accuracy_score(y_test, y_pred), 2)
print("Precisión del modelo:", accuracy)

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.77
Matriz de confusión:
[[15  2  0]
 [ 2  9  0]
 [ 0  3  0]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        17
           1       0.64      0.82      0.72        11
           2       0.00      0.00      0.00         3

    accuracy                           0.77        31
   macro avg       0.51      0.57      0.53        31
weighted avg       0.71      0.77      0.74        31



c:\Users\aemer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aemer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aemer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Podemos hacer una búsqueda aleatoria de hiperparámetros:

In [10]:
parametros = {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              'max_depth': [None, 3, 5, 10, 15, 20, 25, 30, 35, 40]}
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

modelo = RandomizedSearchCV(modelo_base, parametros, cv=k_fold, scoring='accuracy', n_iter=10, random_state=42)
modelo

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(random_state=42),
                   param_distributions={'max_depth': [None, 3, 5, 10, 15, 20,
                                                      25, 30, 35, 40],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]},
                   random_state=42, scoring='accuracy')

Realizamos la búsqueda de hiperparámetros:

In [11]:
# Definir los k-folds
modelo.fit(X_train, y_train)

# Mostrar los resultados
pd.DataFrame(modelo.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.030254,0.000795,0.002799,0.000245,40,35,"{'n_estimators': 40, 'max_depth': 35}",0.666667,0.875000,0.625000,0.875000,0.625000,0.733333,0.116667,3
1,0.029103,0.000373,0.002800,0.000245,40,20,"{'n_estimators': 40, 'max_depth': 20}",0.666667,0.875000,0.625000,0.875000,0.625000,0.733333,0.116667,3
2,0.008104,0.000198,0.001400,0.000200,10,30,"{'n_estimators': 10, 'max_depth': 30}",0.666667,0.833333,0.708333,0.791667,0.583333,0.716667,0.088976,7
3,0.043307,0.000409,0.003700,0.000245,60,15,"{'n_estimators': 60, 'max_depth': 15}",0.666667,0.875000,0.625000,0.875000,0.583333,0.725000,0.125277,5
4,0.035907,0.000199,0.003300,0.000245,50,15,"{'n_estimators': 50, 'max_depth': 15}",0.666667,0.833333,0.666667,0.875000,0.583333,0.725000,0.110554,5
5,0.071210,0.000924,0.005503,0.000320,100,10,"{'n_estimators': 100, 'max_depth': 10}",0.708333,0.875000,0.666667,0.875000,0.583333,0.741667,0.116070,1
6,0.021906,0.000189,0.002300,0.000245,30,5,"{'n_estimators': 30, 'max_depth': 5}",0.708333,0.750000,0.583333,0.833333,0.625000,0.700000,0.088976,10
7,0.008204,0.000250,0.001302,0.000243,10,35,"{'n_estimators': 10, 'max_depth': 35}",0.666667,0.833333,0.708333,0.791667,0.583333,0.716667,0.088976,7
8,0.008100,0.000374,0.001300,0.000245,10,3,"{'n_estimators': 10, 'max_depth': 3}",0.708333,0.916667,0.625000,0.833333,0.625000,0.741667,0.116070,1
9,0.008305,0.000242,0.001200,0.000245,10,None,"{'n_estimators': 10, 'max_depth': None}",0.666667,0.833333,0.708333,0.791667,0.583333,0.716667,0.088976,7


Vemos los mejores hiperparámetros:

In [12]:
# Mejores parámetros
print("Mejores parámetros", modelo.best_params_)
print("Mejor score", modelo.best_score_)

Mejores parámetros {'n_estimators': 100, 'max_depth': 10}
Mejor score 0.7416666666666667


Realizamos la predicción:

In [13]:
# Realizar predicciones y evaluar el modelo
y_pred = modelo.predict(X_test)

# Precisión del modelo con dos decimales
accuracy = round(accuracy_score(y_test, y_pred), 2)
print("Precisión del modelo:", accuracy)

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.71
Matriz de confusión:
[[15  2  0]
 [ 4  7  0]
 [ 0  3  0]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.79      0.88      0.83        17
           1       0.58      0.64      0.61        11
           2       0.00      0.00      0.00         3

    accuracy                           0.71        31
   macro avg       0.46      0.51      0.48        31
weighted avg       0.64      0.71      0.67        31



c:\Users\aemer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aemer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aemer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,